In [18]:
!pip install -U pyupbit
!pip install PublicDataReader
!pip install quart
! pip install Flask

In [ ]:
Python 3.8.2 64-bit('env':venv)

In [ ]:
test_duplicate_password()

In [ ]:
# Flask의 비동기 모드를 활성화 필요 조건
!pip install flask>=2.0

#애플리케이션을 생성할 때, async_mode를 활성화합니다.

In [2]:
from flask import Flask, render_template, request
from PublicDataReader import TransactionPrice
import PublicDataReader as pdr

service_key = "gheSnUnob3mSKc5nus5HfTB5tebwjYj14Y28AM9Fw%2F%2BcET6pE%2Bmsfo1BY7ot0KX5fsuR7RsLDWZYxzvZ7UVOcg%3D%3D"

app = Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        search = request.form['Search']
        api = TransactionPrice(service_key)

        address = search
        split_address = address.split()

        sigungu_name = split_address[1]
        code = pdr.code_bdong()
        filtered_code = code.loc[(code['시군구명'].str.contains(sigungu_name)) & (code['읍면동명'] == '')]

        # 법정동 코드의 앞 다섯 글자만 가져옵니다.
        law_dong_code = str(filtered_code['법정동코드'].values[0])[:5]

        results = {}

        # 부동산종류
        property_types = ["아파트", "오피스텔", "단독다가구", "연립다세대", "토지", "분양입주권", "공장창고"]
        selected_columns = ['도로명', '법정동', '지번', '아파트', '건축년도', '층', '전용면적']

        for property_type in property_types:
            try:
                df = api.get_data(
                    property_type=property_type,
                    trade_type="매매",
                    sigungu_code=law_dong_code,
                    start_year_month="202201",
                    end_year_month="202212",
                )

                if not df.empty:
                    # 지번이 같은 데이터만 필터링합니다.
                    filtered_df = df[df['지번'] == split_address[3]][selected_columns]

                    if not filtered_df.empty:
                        results[property_type] = filtered_df.to_dict('records')
            except:
                pass

        if not results:
            result_string = "검색 결과가 없습니다."
        else:
            select_options = ""
            for property_type, records in results.items():
                select_options += f"<optgroup label='{property_type}'>"
                for record in records:
                    select_options += f"<option>{record['도로명']} {record['법정동']} {record['지번']} ({record['건축년도']}년, {record['층']}층, {record['전용면적']}㎡)</option>"
                select_options += "</optgroup>"
            result_string = f"<select>{select_options}</select>"

    else:
        result_string = ""
#아래코드에서 검색버튼 누를시 검색중 다른작업을 우선으로 진행해주세요 만들어줘
    return '''
    <!DOCTYPE html>
    <html>
        <head>
            <title>Input Text</title>
        </head>
        <body>
            <h1>부동산 주소검색</h1>
            <form action="/" method="POST">
                <p><input type="text" name="Search" placeholder="시군구 동 지번을 입력하세요"></p>
                <p><input type="submit" value="Create" onclick="document.getElementById('loading').style.display='block'"></p>
            </form>
            <div id="loading" style="display:none;">검색중 다른작업을 우선으로 진행해주세요...</div>
            <div>{}</div>
        </body>
        </html>
        '''.format(result_string)

if __name__ == "__main__":
    app.run(debug=False)






 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Apr/2023 15:14:54] "GET / HTTP/1.1" 200 -
